In [ ]:
#pip install openai==0.28
#pip install -U langchain-community
#pip install panel
#pip install pypdf 
#pip install chromadb

In [64]:
import panel as pn
import param
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

pn.extension()

In [88]:
# Define the helper function to load the database
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings(openai_api_key='sk-proj-YbNGoCDkU3KF1y7jxSFIT3BlbkFJqczEVtiepWVtO62SUg7r')
    # create vector database from data
    db = Chroma.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. 
    llm_model=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key='sk-proj-YbNGoCDkU3KF1y7jxSFIT3BlbkFJqczEVtiepWVtO62SUg7r')
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm_model, 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

In [71]:
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    loaded_file = param.String("/Users/pradhikshasuresh/Documents/Python/Space.pdf")
    
    def __init__(self, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []
        self.qa = load_db(self.loaded_file, "stuff", 4)
    
    def call_load_db(self, event):
        if file_input.value:
            with open("temp.pdf", "wb") as f:
                f.write(file_input.value)
            self.loaded_file = "temp.pdf"
            self.qa = load_db(self.loaded_file, "stuff", 4)
            self.clr_history()
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            return pn.pane.Markdown(f"Loaded Default File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.append((query, result["answer"]))
        self.answer = result['answer'] 
        
        self.panels.append(
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
        )
        self.panels.append(
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        )
        
        inp.value = ''  # clears the input box
        return pn.WidgetBox(*self.panels, scroll=True)

    def clr_history(self, event=None):
        self.chat_history = []
        self.panels = []  # Clear the panels as well
        return pn.pane.Markdown("Chat history cleared.")

In [69]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_load.on_click(cb.call_load_db)
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Bind the convchain method to the input field
conversation = pn.bind(cb.convchain, inp)

# Layout including the file input and load button
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    conversation,
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.Row(file_input, button_load),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatBot')),
    pn.Tabs(('Conversation', tab1)),
    pn.Row(button_clearhistory),
    tab2
)

In [70]:
dashboard.servable()

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False)
            [3] Divider()
    [2] Row
        [0] Button(button_type='warning', name='Clear History')
    [3] Column
        [0] Row
            [0] FileInput(accept='.pdf')
            [1] Button(button_type='primary', name='Load DB')
        [1] Divider()